In [1]:
pip install langchain langchain-chroma langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass

os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Langchain API Key: ")
os.environ["LANGCHAIN_V2_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchain-vectorstores"
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")


Langchain API Key: ········
Anthropic API Key: ········
OpenAI API Key: ········


In [7]:
from langchain.callbacks import LangChainTracer, tracing_v2_enabled
from langchain.callbacks.manager import CallbackManager

In [11]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

tracer = LangChainTracer(project_name="langchain-vectorstores")
callback_manager = CallbackManager([tracer])

llm = ChatOpenAI(model_name="gpt-4", callback_manager=callback_manager)
with tracing_v2_enabled():
    response = llm.invoke("Who is Varun Mayya? Is he relevant as a fair ai enthusiast?")
print(response)

content='Varun Mayya is an entrepreneur, programmer, and author. He is the CEO of Avalon Labs and the co-founder of Jobspire, a recruiting platform that was later acquired. He is also the author of the book "Pyjama Profit," which focuses on freelancing and remote work.\n\nWhile Mayya\'s work is primarily in the tech and startup sectors, there is no specific information suggesting that he is a fair AI enthusiast. His relevance in the AI field would likely be more related to his entrepreneurship and leadership in tech companies that might utilize AI, rather than direct involvement in AI development or research.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 23, 'total_tokens': 146}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e2efd17c-c22b-4a6b-89d6-19f91c0a4aa0-0' usage_metadata={'input_tokens': 23, 'output_tokens': 123, 'total_tokens': 146}


In [12]:
from langchain_core.documents import Document

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)



In [14]:
vectorstore.similarity_sOearch("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [17]:
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [18]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.3747221529483795),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.4829659163951874),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.495543509721756),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.4971568286418915)]

In [20]:
embeddings = OpenAIEmbeddings().embed_query("cat")
vectorstore.similarity_search_by_vector(embeddings)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [21]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [22]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [27]:
import getpass
import os

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", callback_manager=callback_manager)

········


In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.callbacks import LangChainTracer, tracing_v2_enabled
from langchain.callbacks.manager import CallbackManager

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | #llm (this is a runnable)

In [32]:
with tracing_v2_enabled():
    response = rag_chain.invoke("Can Rabbits can mimick humans? if not, who can?")

print(response.content)

Based solely on the provided context, rabbits are not mentioned as being able to mimic humans. However, the context does state that parrots are capable of mimicking human speech. Specifically, it says "Parrots are intelligent birds capable of mimicking human speech."
